[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/srchd/machine-learning/blob/visu/setting-up-visu/plane/visu.ipynb)

In [ ]:
import os
import pickle

import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dropout, Input, Conv2D, AveragePooling2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.optimizers

!wget -nc -P datasets/ https://github.com/srchd/machine-learning/tree/visu/setting-up-visu/plane/datasets/class_data.pkl
!wget -nc -P datasets/ https://github.com/srchd/machine-learning/tree/visu/setting-up-visu/plane/datasets/class_label.pkl


CAT_NAMES = ['aeroplane', 'bird', 'boat', 'bus', 'car', 'cat', 'cow', 'dog', 'horse', 'sheep']

with open('datasets/class_data.pkl', 'rb') as f:
    arr_cl_data = pickle.load(f)

with open('datasets/class_label.pkl', 'rb') as f:
    arr_cl_labs = pickle.load(f)

print(f'Shape of classification image array: {arr_cl_data.shape}')
print(f'Shape of classification label array: {arr_cl_labs.shape}')